# Annotations Implementation

## Setup

In [1]:
import os
import sys

main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(main_dir)
print(main_dir)

import imageio
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from RemoteSensing.changedetection.datasets.make_data_loader import SemanticChangeDetectionDatset, make_data_loader, SemanticChangeDetectionDatset_LandSat
from RemoteSensing.changedetection.utils_func.metrics import Evaluator
from RemoteSensing.changedetection.models.STMambaSCD import STMambaSCD
import RemoteSensing.changedetection.utils_func.lovasz_loss as L
from torch.optim.lr_scheduler import StepLR
from RemoteSensing.changedetection.utils_func.mcd_utils import accuracy, SCDD_eval_all, AverageMeter
from RemoteSensing.changedetection.utils_func.loss import contrastive_loss, ce2_dice1, ce2_dice1_multiclass, SeK_Loss, SEK_loss_from_eval
from RemoteSensing.changedetection.configs.config import get_config
import RemoteSensing.changedetection.datasets.imutils as imutils
import torchvision.transforms.functional as TF

from dotenv import load_dotenv

load_dotenv()
def getPath(env_path):
    return os.path.expanduser(os.getenv(env_path))


torch.cuda.set_device(0)

/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/RemoteSensing/classification/models/vmamba.py:252: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/RemoteSensing/classification/models/vmamba.py:260: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/RemoteSensing/classification/models/vmamba.py:275: FutureWarning: `torch.cuda.amp.custom_fw

In [2]:
VSSM_MODEL_PATH = getPath('VSSMBASEPATH')
SECOND_DATASET_PATH = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Datasets/SECOND/')


SECOND_TRAIN_DATASET_PATH = os.path.join(SECOND_DATASET_PATH, 'train')
SECOND_TEST_DATASET_PATH = os.path.join(SECOND_DATASET_PATH, 'test')
SECOND_TRAIN_DATA_LIST_PATH = os.path.join(SECOND_DATASET_PATH, 'train.txt')
SECOND_TEST_DATA_LIST_PATH = os.path.join(SECOND_DATASET_PATH, 'test.txt')

torch.cuda.set_device(1)

configs_path = os.path.join(main_dir, 'RemoteSensing/changedetection/configs/vssm1/vssm_base_224.yaml')

model_path = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/JOURNAL/')

MAMBAFCS = os.path.abspath('/storage/scratch3/buddhiw-change-detection/Mamba/JOURNAL/SECOND_without_dice_without_DIFF_without_freq_2/15000_model_0.240.pth')

SAVE_IMGS_PATH = os.path.join(main_dir, 'RemoteSensing/annotations/Images/FFT/')

train_data_list = []
with open(SECOND_TRAIN_DATA_LIST_PATH, 'r') as f:
    for line in f:
        train_data_list.append(line.strip())

test_data_list = []
with open(SECOND_TEST_DATA_LIST_PATH, 'r') as f:
    for line in f:
        test_data_list.append(line.strip())

In [3]:
class ARGS:
    def __init__(self):
        self.cfg = configs_path
        self.opts = None
        self.pretrained_weight_path = VSSM_MODEL_PATH
        self.dataset = 'SECOND'
        self.type = 'train'
        self.train_dataset_path = SECOND_TRAIN_DATASET_PATH

        self.test_dataset_path = SECOND_TEST_DATASET_PATH

        
        self.shuffle = True
        self.batch_size = 4
        self.crop_size = 512
        self.train_data_name_list = train_data_list
        self.test_data_name_list = test_data_list
        self.start_iter = 0
        self.cuda = True
        self.max_iters = 800000
        self.model_type = 'MambaSCD_base'
        self.model_param_path = model_path

        self.resume = MAMBAFCS

        self.learning_rate = 1e-4
        self.momentum = 0.9
        self.weight_decay = 5e-4
        self.num_classes = 7

args = ARGS()

In [4]:
config = get_config(args)

train_data_loader = make_data_loader(args)

deep_model = STMambaSCD(
    output_cd = 2, 
    output_clf = args.num_classes,
    pretrained=args.pretrained_weight_path,
    patch_size=config.MODEL.VSSM.PATCH_SIZE, 
    in_chans=config.MODEL.VSSM.IN_CHANS, 
    num_classes=config.MODEL.NUM_CLASSES, 
    depths=config.MODEL.VSSM.DEPTHS, 
    dims=config.MODEL.VSSM.EMBED_DIM, 
    # ===================
    ssm_d_state=config.MODEL.VSSM.SSM_D_STATE,
    ssm_ratio=config.MODEL.VSSM.SSM_RATIO,
    ssm_rank_ratio=config.MODEL.VSSM.SSM_RANK_RATIO,
    ssm_dt_rank=("auto" if config.MODEL.VSSM.SSM_DT_RANK == "auto" else int(config.MODEL.VSSM.SSM_DT_RANK)),
    ssm_act_layer=config.MODEL.VSSM.SSM_ACT_LAYER,
    ssm_conv=config.MODEL.VSSM.SSM_CONV,
    ssm_conv_bias=config.MODEL.VSSM.SSM_CONV_BIAS,
    ssm_drop_rate=config.MODEL.VSSM.SSM_DROP_RATE,
    ssm_init=config.MODEL.VSSM.SSM_INIT,
    forward_type=config.MODEL.VSSM.SSM_FORWARDTYPE,
    # ===================
    mlp_ratio=config.MODEL.VSSM.MLP_RATIO,
    mlp_act_layer=config.MODEL.VSSM.MLP_ACT_LAYER,
    mlp_drop_rate=config.MODEL.VSSM.MLP_DROP_RATE,
    # ===================
    drop_path_rate=config.MODEL.DROP_PATH_RATE,
    patch_norm=config.MODEL.VSSM.PATCH_NORM,
    norm_layer=config.MODEL.VSSM.NORM_LAYER,
    downsample_version=config.MODEL.VSSM.DOWNSAMPLE,
    patchembed_version=config.MODEL.VSSM.PATCHEMBED,
    gmlp=config.MODEL.VSSM.GMLP,
    use_checkpoint=config.TRAIN.USE_CHECKPOINT,
    ) 

deep_model = deep_model.cuda()

=> merge config from /storage/scratch3/buddhiw-change-detection/ChangeDetection/CDMamba/RemoteSensing/changedetection/configs/vssm1/vssm_base_224.yaml
Successfully load ckpt /storage/scratch3/buddhiw-change-detection/ChangeDetection/VSSModels/pretrained/vssm_base_0229_ckpt_epoch_237.pth
_IncompatibleKeys(missing_keys=['outnorm0.weight', 'outnorm0.bias', 'outnorm1.weight', 'outnorm1.bias', 'outnorm2.weight', 'outnorm2.bias', 'outnorm3.weight', 'outnorm3.bias'], unexpected_keys=['classifier.norm.weight', 'classifier.norm.bias', 'classifier.head.weight', 'classifier.head.bias'])
False


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [5]:
deep_model.load_state_dict(torch.load(args.resume))


deep_model.cuda()
deep_model.eval()

STMambaSCD(
  (encoder): Backbone_VSSM(
    (patch_embed): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): Permute()
      (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (3): Permute()
      (4): GELU(approximate='none')
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (6): Permute()
      (7): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (layers): ModuleList(
      (0): Sequential(
        (blocks): Sequential(
          (0): VSSBlock(
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (op): SS2D(
              (out_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (in_proj): Linear(in_features=128, out_features=256, bias=False)
              (act): SiLU()
              (conv2d): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=256, bias=False)
              (out_proj): Linear(in_

In [6]:
def img_loader(path):
    img = np.array(imageio.imread(path), np.float32)
    return img

# Ground Truth Annotations

In [7]:
ori_label_value_dict = {
    'background': (255, 255, 255),
    'low_vegetation': (0, 128, 0),
    'nvg_surface': (128, 128, 128),
    'tree': (0, 255, 0),
    'water': (0, 0, 255),
    'Building': (128, 0, 0),
    'Playground': (255, 0, 0)
}

target_label_value_dict = {
    'background': 0,
    'low_vegetation': 1,
    'nvg_surface': 2,
    'tree': 3,
    'water': 4,
    'Building': 5,
    'Playground': 6
}

def map_labels_to_colors(labels, ori_label_value_dict, target_label_value_dict):
    # Reverse the target_label_value_dict to get a mapping from target labels to original labels
    target_to_ori = {v: k for k, v in target_label_value_dict.items()}
    
    # Initialize an empty 3D array for the color-mapped labels
    H, W = labels.shape
    color_mapped_labels = np.zeros((H, W, 3), dtype=np.uint8)
    
    for target_label, ori_label in target_to_ori.items():
        # Find where the label matches the current target label
        mask = labels == target_label
        
        # Map these locations to the corresponding color value
        color_mapped_labels[mask] = ori_label_value_dict[ori_label]
    
    return color_mapped_labels

In [8]:
def annotate_image(image_number):
    pre_image_path = os.path.join(SECOND_TEST_DATASET_PATH,"T1",f"{image_number}")
    post_image_path = os.path.join(SECOND_TEST_DATASET_PATH,"T2",f"{image_number}")
    GT_CD_image_path = os.path.join(SECOND_TEST_DATASET_PATH,"GT_CD",f"{image_number}")
    GT_T1_image_path = os.path.join(SECOND_TEST_DATASET_PATH,"GT_T1_COLORED",f"{image_number}")
    GT_T2_image_path = os.path.join(SECOND_TEST_DATASET_PATH,"GT_T2_COLORED",f"{image_number}")

    image_number = image_number[:-4] 

    pre_image = img_loader(pre_image_path)
    post_image = img_loader(post_image_path)
    GT_CD_image = img_loader(GT_CD_image_path)
    GT_T1_image = img_loader(GT_T1_image_path)
    GT_T2_image = img_loader(GT_T2_image_path)

    GT_CD_image = GT_CD_image

    pre_img = imutils.normalize_img(pre_image)  # imagenet normalization
    pre_img = np.transpose(pre_img, (2, 0, 1))

    post_img = imutils.normalize_img(post_image)  # imagenet normalization
    post_img = np.transpose(post_img, (2, 0, 1))

    GT_CD_image = np.asarray(GT_CD_image)
    GT_T1_image = np.asarray(GT_T1_image)
    GT_T2_image = np.asarray(GT_T2_image)

    # save original images
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pre.png"), pre_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_post.png"), post_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_gt_cd.png"), GT_CD_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_gt_t1.png"), GT_T1_image.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_gt_t2.png"), GT_T2_image.astype(np.uint8))

    pre_tensor = TF.to_tensor(imutils.normalize_img(pre_image)).unsqueeze(0).cuda()
    post_tensor = TF.to_tensor(imutils.normalize_img(post_image)).unsqueeze(0).cuda()
    label_cd = TF.to_tensor(GT_CD_image).cuda().long()
    label_t1 = TF.to_tensor(GT_T1_image).cuda().long()
    label_t2 = TF.to_tensor(GT_T2_image).cuda().long()

    with torch.no_grad():
        output_cd, output_t1, output_t2 = deep_model(pre_tensor, post_tensor)

        change_mask = torch.argmax(output_cd, axis=1)

        preds_A = torch.argmax(output_t1, dim = 1)
        preds_B = torch.argmax(output_t2, dim = 1)

        preds_A = (preds_A*change_mask.squeeze().long()).cpu().numpy()
        preds_B = (preds_B*change_mask.squeeze().long()).cpu().numpy()

    preds_A = map_labels_to_colors(np.squeeze(preds_A), ori_label_value_dict=ori_label_value_dict, target_label_value_dict=target_label_value_dict)
    preds_B = map_labels_to_colors(np.squeeze(preds_B), ori_label_value_dict=ori_label_value_dict, target_label_value_dict=target_label_value_dict)

    change_mask = change_mask * 255  # Convert to uint8 for saving

    # save predicted images
    imageio.imwrite(
                os.path.join(SAVE_IMGS_PATH, f"{image_number}_pred_cd.png"),
                change_mask.squeeze(0).cpu().numpy().astype(np.uint8)
            )
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pred_t1.png"), preds_A.astype(np.uint8))
    imageio.imwrite(os.path.join(SAVE_IMGS_PATH, f"{image_number}_pred_t2.png"), preds_B.astype(np.uint8))

In [30]:
annotate_image("08931.png")

/tmp/ipykernel_2624718/3105635309.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = np.array(imageio.imread(path), np.float32)
